In [4]:
import pandas as pd
import numpy as np
from typing import Tuple
import matplotlib.pyplot as plt

from scipy.optimize import minimize

import sys
sys.path.insert(0, '../scripts/')
import utils as utl

In [18]:
df = pd.read_csv('../data/1.raw/Stars.csv')#, parse_dates=['DateTime'],index_col=['DateTime'])

X_cols = ['Temperature', 'L', 'R', 'A_M']
y_col = 'Type'

X = df[X_cols]
for c in X.columns:
    X[c] = utl.min_max_scaling(X[c])[0]

y = df[y_col]

C:\Users\an2fe\AppData\Local\Temp\ipykernel_13964\3294099598.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[c] = utl.min_max_scaling(X[c])[0]


In [19]:
y_res = pd.DataFrame()
for cat in y.unique():
    y_np = y.copy().to_numpy()
    y_np[y_np!=cat]=-1
    y_np[y_np==cat]=1
    y_np[y_np==-1]=0
    y_res[cat] = y_np

In [38]:
X_mod = X.copy()
ones_aux = np.ones(X.shape[0])
X_mod['bias_col'] = ones_aux
cols = X_mod.columns
cols_reordered = [cols[i-1] for i,x in enumerate(cols)]
X_mod = X_mod[cols_reordered]
X_np = X_mod.to_numpy()
m = len(X_mod)



# Inicializes data
J = np.ones(len(y_res.columns))
w_df = pd.DataFrame()
for i,val in enumerate(y_res.columns):
    w_df[val] = 0.5*np.ones(X.shape[1]+1)
w = w_df.to_numpy()
y_c = y_res.to_numpy()

# Calculates cost function for each class
for i,val in enumerate(y_res.columns):    
    y_pred = (w[:,0]*X_np).sum(axis=1)
    h = 1/(1 + np.exp(y_pred))
    J[i] = - (1/m)*np.sum(y_c[:,i]*np.log(h) + (1-y_c[:,i])*np.log(1-h))
J

array([0.48097969, 0.46852552, 0.49193453, 0.46609313, 0.47744671,
       0.51725823])